In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
# GPU 사용 설정
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = '1'

In [3]:
from tensorflow import keras
(train_input, train_target),(test_input, test_target) = keras.datasets.fashion_mnist.load_data()

In [4]:
# 일렬로 만드는건 CNN에선 아예 안쓰지만 일단 표준화후 일렬로 만들자
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28*28)

In [5]:
# train과 valid 분리
from sklearn.model_selection import train_test_split
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size=0.2, random_state=42
)

### Layer를 추가하는 방법 -1
- 두번째로 많이 쓰는 방법

In [6]:
# Layer만들기
# Dense :  밀집층(다 연결되어 있어서)
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)) # 은닉층이라 분류가 아니야
dense2 = keras.layers.Dense(10, activation='softmax') # 출력층  

In [7]:
# 전체 모델에 층을 추가
model= keras.Sequential([dense1, dense2])

In [8]:
# Summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


- Param 가 w(가중치) 갯수
- 1010 =>  100 * 10 + 10 => 절편의 수도 합쳐진수

---
### Layer를 추가하는 방법 -2
- 층이 데이터에 따라 다르게 생길수 있어서 if문을 넣을수가 없어서 힘듦

In [9]:
model = keras.Sequential(
    [
        keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'),
        keras.layers.Dense(10, activation='softmax', name='output') 
    ],name = '패션 Mnist 모델'
)

In [10]:
# Summary
model.summary()

Model: "패션 Mnist 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
### Layer를 추가하는 방법 -3
- 주로 쓰는 방법임
- if 문의 사용이 가능함
- 갈라치기가 알파고래!

In [11]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation='sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation='softmax'))

In [12]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 100)               78500     
                                                                 
 dense_3 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
# 모델 훈련

In [13]:
model.compile(
    loss='sparse_categorical_crossentropy',
    metrics='accuracy'
)

model.fit(train_scaled, train_target, epochs=5)

Epoch 1/5


2022-07-20 14:49:28.748702: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1500/1500 [==============================] - 1s 708us/step - loss: 0.5608 - accuracy: 0.8119
Epoch 2/5
1500/1500 [==============================] - 1s 668us/step - loss: 0.4070 - accuracy: 0.8526
Epoch 3/5
1500/1500 [==============================] - 1s 688us/step - loss: 0.3727 - accuracy: 0.8651
Epoch 4/5
1500/1500 [==============================] - 1s 710us/step - loss: 0.3512 - accuracy: 0.8732
Epoch 5/5
1500/1500 [==============================] - 1s 778us/step - loss: 0.3333 - accuracy: 0.8796


- Sigmoid함수는 숫자가 무한대로 커지게 되면 데이터의 차이가 거의 없다는 단점이 있다.
- 이를 해결한것이 Relu함수인데 인공신경망에 기여한 바가 크며 음수는 모든 0으로 양수는 항상 최대값을 출력하는 함수

In [17]:
# 입력층을 단순화 하게
model = keras.Sequential()
# Flatten  을 사용하는게 제일 편함
model.add(keras.layers.Flatten(input_shape=(28, 28))) # 입력층을 2차원데이터인데 이걸 바꿔주지 않고 원래대로 사용한다.
                                                      #  차원을 알아서 바꿔줌
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

In [18]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


---
## Optimizer(최적화 알고리즘)
- 훈련시에 Optimizer를 사용하여 최적화 알고리즘을 구성한다.
- 보통 훈련시 사용하는 것이 확률적 경사하강법 optimizer 이다.

In [20]:
# 1) 적용방법 (확률적 경사 하강법) -1
model.compile(optimizer='sgd', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [21]:
# 2) 적용방법 (확률적 경사 하강법) -2
sgd = keras.optimizers.SGD()
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy', metrics='accuracy')

In [22]:
# 3) 적용방법 (확률적 경사 하강법) + 학습률 변경
# SGD의 learning rate의 기본값은 0.01
sgd = keras.optimizers.SGD(learning_rate=0.1)
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy', metrics='accuracy')

In [ ]:
# 적용방법 (Momentum)

sgd = keras.optimizers.SGD(momentum=0.9, nesterov=True)

In [23]:
# 적용방법 (Adagrad)
adagrad = keras.optimizers.Adagrad()
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy', metrics='accuracy')

# 또는
# model.compile(optimizer='adagrad', loss='sparse_categorical_crossentropy', metrics='accuracy')

In [24]:
# 적용방법 (Rmsprop)

rmsprop = keras.optimizers.RMSprop()
model.compile(optimizer=sgd,loss='sparse_categorical_crossentropy', metrics='accuracy')